# Lab Assignment 1

Student name: Cornelius Adejoro

## Notebook version

This notebook includes all the codes in the codebase of lab assignment 1. Completing and submitting this script is equivalent to submitting the codebase. Please note that your submitted script should include errorless cell outputs that contain necessary information that proves you have successfully run the notebook in your own directory.

You can choose to (1) run this notebook locally on your end or (2) run this notebook on colab. For the former, you will need to download the dataset to your device that resembles the instructions for the codebase. For the latter, **you will need to upload the dataset to your Google Drive** account, and connect your colab notebook to your Google Drive. Then, go to "File->Save a copy in Drive" to create a copy you can edit.


#### Colab (if applicable)

If you are running this script on colab, uncomment and run the cell below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Note that the Google Drive directory has the root `/content/drive/`. For instance, my directory to the dataset is `'/content/drive/My Drive/Courses/CSCI 5922/CSCI 5922 SP25/Demo/MNIST/'`.

### mnist.py

In [ ]:
#Original source: https://www.kaggle.com/code/hojjatk/read-mnist-dataset
#It has been modified for ease of use w/ pytorch

#You do NOT need to modify ANY code in this file!

import numpy as np
import struct
from array import array
import torch

class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        n = 60000 if "train" in images_filepath else 10000
        labels = torch.zeros((n, 10))
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            l = torch.tensor(array("B", file.read())).unsqueeze(-1)
            l = torch.concatenate((torch.arange(0, n).unsqueeze(-1), l), dim = 1).type(torch.int32)
            labels[l[:,0], l[:,1]] = 1

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = torch.zeros((n, 28**2))
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            #img = img.reshape(28, 28)
            images[i, :] = torch.tensor(img)

        return images, labels

    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

### activations.py

In [ ]:
import torch

class ReLU():
  #Complete this class
    def forward(self, x: torch.tensor) -> torch.tensor:
        return torch.maximum(x, torch.tensor(0.0))

    def backward(self, delta: torch.tensor, x: torch.tensor) -> torch.tensor:
        return delta * (x > 0).float()


class LeakyReLU():
  #Complete this class
    def forward(self, x: torch.tensor) -> torch.tensor:
        return torch.maximum(0.1 * x, x)

    def backward(self, delta: torch.tensor, x: torch.tensor) -> torch.tensor:
        return delta * torch.where(x > 0, torch.tensor(1.0), torch.tensor(0.1))

### framework.py

In [ ]:
import torch
import numpy as np
import tqdm

class MLP:
    '''
    This class implements a generic MLP learning framework. The core structure is provided,
    but key functions such as initialize(), forward(), backward(), and TrainMLP() have been completed.
    '''
    def __init__(self, layer_sizes: list[int]):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        self.weights = []
        self.biases = []
        self.features = []

        # Hyperparameters with default values
        self.learning_rate = 1e-6
        self.batch_size = 512
        self.epoch = 25
        self.activation_function = ReLU()

    def set_hp(self, lr: float, bs: int, activation: object) -> None:
        self.learning_rate = lr
        self.batch_size = bs
        self.activation_function = activation

    def initialize(self) -> None:
        # Initialize weights with uniform distribution +/- sqrt(6 / (d_in + d_out))
        for i in range(self.num_layers):
            d_in, d_out = self.layer_sizes[i], self.layer_sizes[i + 1]
            bound = np.sqrt(6 / (d_in + d_out))
            self.weights.append(torch.empty((d_in, d_out)).uniform_(-bound, bound))
            #self.weights.append(torch.rand((d_in, d_out)) * 2 * bound - bound)
            self.biases.append(torch.zeros(1, d_out))

    def forward(self, x: torch.tensor) -> torch.tensor:
        # Forward propagation
        self.features = [x]
        for i in range(self.num_layers):
            x = x @ self.weights[i] + self.biases[i]
            x = self.activation_function.forward(x)
            self.features.append(x)
        logits = self.features[-1]
        softmax_x = torch.exp(logits) / torch.sum(torch.exp(logits), dim=1, keepdim=True)
        return softmax_x

    def backward(self, delta: torch.tensor) -> None:
      for i in reversed(range(self.num_layers)):
          # Gradient of weights and biases
          grad_w = self.features[i].T @ delta
          grad_b = delta.mean(dim=0)

          # Update weights and biases using gradient descent
          self.weights[i] -= self.learning_rate * grad_w
          self.biases[i] -= self.learning_rate * grad_b

          # Compute delta for the previous layer (only if it's not the input layer)
          if i > 0:
              # grad_w = self.weights[i]
              # delta = delta @ grad_w.T
              # delta = self.activation_function.backward(delta, self.features[i - 1])

              delta = self.activation_function.backward( (delta @ self.weights[i].T),(self.features[i]))

              # grad_b = delta.mean(dim=0)
              # delta = (delta @ self.weights[i].T) * (self.features[i] > 0).float()  # Apply ReLU derivative

def TrainMLP(model: MLP, x_train: torch.tensor, y_train: torch.tensor) -> MLP:
    bs = model.batch_size
    N = x_train.shape[0]
    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0  # Total loss

    for i in tqdm.tqdm(range(N // bs)):
        x = x_train[idx[i * bs:(i + 1) * bs], ...]
        y = y_train[idx[i * bs:(i + 1) * bs], ...]

        # Forward pass
        y_hat = model.forward(x)

        # Compute cross-entropy loss (convert one-hot to class indices)
        target = torch.argmax(y, dim=1)
        l = -torch.sum(y * torch.log(y_hat))
        L += l.item()

        # Backward pass
        delta = y_hat - y  # Gradient for softmax + cross-entropy
        model.backward(delta)

    print("Train Loss:", L / ((N // bs) * bs))

def TestMLP(model: MLP, x_test: torch.tensor, y_test: torch.tensor) -> tuple[float, float]:
    bs = model.batch_size
    N = x_test.shape[0]

    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0
    A = 0

    for i in tqdm.tqdm(range(N // bs)):
        x = x_test[idx[i * bs:(i + 1) * bs], ...]
        y = y_test[idx[i * bs:(i + 1) * bs], ...]

        y_hat = model.forward(x)
        target = torch.argmax(y, dim=1)

        # Cross-entropy loss
        l = torch.nn.functional.cross_entropy(y_hat, target)
        L += l.item()

        # Accuracy calculation
        predictions = torch.argmax(y_hat, dim=1)
        A += torch.sum(predictions == target).item()

    print("Test Loss:", L / ((N // bs) * bs), "Test Accuracy: {:.2f}%".format(100 * A / N))

def normalize_mnist() -> tuple[torch.tensor, torch.tensor, torch.tensor, torch.tensor]:
    base_path = "/content/drive/MyDrive/Neural_Networks/Data/"

    mnist = MnistDataloader(base_path + "train-images.idx3-ubyte", base_path + "train-labels.idx1-ubyte",
                            base_path + "t10k-images.idx3-ubyte", base_path + "t10k-labels.idx1-ubyte")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_mean = torch.mean(x_train, dim=0, keepdim=True)
    x_std = torch.std(x_train, dim=0, keepdim=True)

    x_train -= x_mean
    x_train /= x_std
    x_train[x_train != x_train] = 0

    x_test -= x_mean
    x_test /= x_std
    x_test[x_test != x_test] = 0

    return x_train, y_train, x_test, y_test

def main():
    x_train, y_train, x_test, y_test = normalize_mnist()

    model = MLP([784, 256, 10])
    model.initialize()
    model.set_hp(lr=1e-6, bs=512, activation=ReLU())

    E = 25
    for _ in range(E):
        TrainMLP(model, x_train, y_train)
        TestMLP(model, x_test, y_test)

if __name__ == "__main__":
    main()


100%|██████████| 117/117 [00:02<00:00, 51.50it/s]


Train Loss: 2.382489379654583


100%|██████████| 19/19 [00:00<00:00, 86.11it/s]


Test Loss: nan Test Accuracy: 18.25%


100%|██████████| 117/117 [00:02<00:00, 57.37it/s]


...

Test Loss: nan Test Accuracy: 81.51%


100%|██████████| 117/117 [00:01<00:00, 91.14it/s]


Train Loss: 0.6673054975322169


100%|██████████| 19/19 [00:00<00:00, 170.68it/s]

Test Loss: nan Test Accuracy: 81.87%


Part Two

In [ ]:
import torch
import numpy as np
import tqdm

class MLP:
    '''
    This class implements a generic MLP learning framework. The core structure is provided,
    but key functions such as initialize(), forward(), backward(), and TrainMLP() have been completed.
    '''
    def __init__(self, layer_sizes: list[int]):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        self.weights = []
        self.biases = []
        self.features = []

        # Hyperparameters with default values
        self.learning_rate = 1e-6
        self.batch_size = 512
        self.epoch = 25
        self.activation_function = ReLU()

    def set_hp(self, lr: float, bs: int, activation: object) -> None:
        self.learning_rate = lr
        self.batch_size = bs
        self.activation_function = activation

    def initialize(self) -> None:
        # Initialize weights with uniform distribution +/- sqrt(6 / (d_in + d_out))
        for i in range(self.num_layers):
            d_in, d_out = self.layer_sizes[i], self.layer_sizes[i + 1]
            bound = np.sqrt(6 / (d_in + d_out))
            self.weights.append(torch.empty((d_in, d_out)).uniform_(-bound, bound))
            #self.weights.append(torch.rand((d_in, d_out)) * 2 * bound - bound)
            self.biases.append(torch.zeros(1, d_out))

    def forward(self, x: torch.tensor) -> torch.tensor:
        # Forward propagation
        self.features = [x]
        for i in range(self.num_layers):
            x = x @ self.weights[i] + self.biases[i]
            x = self.activation_function.forward(x)
            self.features.append(x)
        logits = self.features[-1]
        softmax_x = torch.exp(logits) / torch.sum(torch.exp(logits), dim=1, keepdim=True)
        return softmax_x

    def backward(self, delta: torch.tensor) -> None:
      for i in reversed(range(self.num_layers)):
          # Gradient of weights and biases
          grad_w = self.features[i].T @ delta
          grad_b = delta.mean(dim=0)

          # Update weights and biases using gradient descent
          self.weights[i] -= self.learning_rate * grad_w
          self.biases[i] -= self.learning_rate * grad_b

          # Compute delta for the previous layer (only if it's not the input layer)
          if i > 0:
              # grad_w = self.weights[i]
              # delta = delta @ grad_w.T
              # delta = self.activation_function.backward(delta, self.features[i - 1])

              delta = self.activation_function.backward( (delta @ self.weights[i].T),(self.features[i]))

              # grad_b = delta.mean(dim=0)
              # delta = (delta @ self.weights[i].T) * (self.features[i] > 0).float()  # Apply ReLU derivative

def TrainMLP(model: MLP, x_train: torch.tensor, y_train: torch.tensor) -> MLP:
    bs = model.batch_size
    N = x_train.shape[0]
    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0  # Total loss

    for i in tqdm.tqdm(range(N // bs)):
        x = x_train[idx[i * bs:(i + 1) * bs], ...]
        y = y_train[idx[i * bs:(i + 1) * bs], ...]

        # Forward pass
        y_hat = model.forward(x)

        # Compute cross-entropy loss (convert one-hot to class indices)
        target = torch.argmax(y, dim=1)
        l = -torch.sum(y * torch.log(y_hat))
        L += l.item()

        # Backward pass
        delta = y_hat - y  # Gradient for softmax + cross-entropy
        model.backward(delta)

    print("Train Loss:", L / ((N // bs) * bs))

def TestMLP(model: MLP, x_test: torch.tensor, y_test: torch.tensor) -> tuple[float, float]:
    bs = model.batch_size
    N = x_test.shape[0]

    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0
    A = 0

    for i in tqdm.tqdm(range(N // bs)):
        x = x_test[idx[i * bs:(i + 1) * bs], ...]
        y = y_test[idx[i * bs:(i + 1) * bs], ...]

        y_hat = model.forward(x)
        target = torch.argmax(y, dim=1)

        # Cross-entropy loss
        l = torch.nn.functional.cross_entropy(y_hat, target)
        L += l.item()

        # Accuracy calculation
        predictions = torch.argmax(y_hat, dim=1)
        A += torch.sum(predictions == target).item()

    print("Test Loss:", L / ((N // bs) * bs), "Test Accuracy: {:.2f}%".format(100 * A / N))

def normalize_mnist() -> tuple[torch.tensor, torch.tensor, torch.tensor, torch.tensor]:
    base_path = "/content/drive/MyDrive/Neural_Networks/Data/"

    mnist = MnistDataloader(base_path + "train-images.idx3-ubyte", base_path + "train-labels.idx1-ubyte",
                            base_path + "t10k-images.idx3-ubyte", base_path + "t10k-labels.idx1-ubyte")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_mean = torch.mean(x_train, dim=0, keepdim=True)
    x_std = torch.std(x_train, dim=0, keepdim=True)

    x_train -= x_mean
    x_train /= x_std
    x_train[x_train != x_train] = 0

    x_test -= x_mean
    x_test /= x_std
    x_test[x_test != x_test] = 0

    return x_train, y_train, x_test, y_test

def main():
    x_train, y_train, x_test, y_test = normalize_mnist()

    model = MLP([784, 10, 10])
    model.initialize()
    model.set_hp(lr=1e-6, bs=512, activation=ReLU())

    E = 25
    for _ in range(E):
        TrainMLP(model, x_train, y_train)
        TestMLP(model, x_test, y_test)

if __name__ == "__main__":
    main()


100%|██████████| 117/117 [00:00<00:00, 435.06it/s]


Train Loss: 2.302205695046319


100%|██████████| 19/19 [00:00<00:00, 506.85it/s]


Test Loss: nan Test Accuracy: 16.06%


100%|██████████| 117/117 [00:00<00:00, 434.26it/s]


...

Test Loss: nan Test Accuracy: 62.23%


100%|██████████| 117/117 [00:00<00:00, 408.94it/s]


Train Loss: 1.3024882017037807


100%|██████████| 19/19 [00:00<00:00, 547.67it/s]

Test Loss: nan Test Accuracy: 62.83%


# L1 Normaliation Implementation

In [ ]:
import torch
import numpy as np
import tqdm

class MLP:
    '''
    This class implements a generic MLP learning framework. The core structure is provided,
    but key functions such as initialize(), forward(), backward(), and TrainMLP() have been completed.
    '''
    def __init__(self, layer_sizes: list[int]):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        self.weights = []
        self.biases = []
        self.features = []

        # Hyperparameters with default values
        self.learning_rate = 1e-6
        self.batch_size = 512
        self.epoch = 25
        self.activation_function = ReLU()

    def set_hp(self, lr: float, bs: int, activation: object, l1_reg: float = 0.0) -> None: # Added l1_reg with a default value
        self.learning_rate = lr
        self.batch_size = bs
        self.activation_function = activation
        self.l1_reg = l1_reg # Assign l1_reg to the model

    def initialize(self) -> None:
        # Initialize weights with uniform distribution +/- sqrt(6 / (d_in + d_out))
        for i in range(self.num_layers):
            d_in, d_out = self.layer_sizes[i], self.layer_sizes[i + 1]
            bound = np.sqrt(6 / (d_in + d_out))
            self.weights.append(torch.empty((d_in, d_out)).uniform_(-bound, bound))
            #self.weights.append(torch.rand((d_in, d_out)) * 2 * bound - bound)
            self.biases.append(torch.zeros(1, d_out))

    def forward(self, x: torch.tensor) -> torch.tensor:
        # Forward propagation
        self.features = [x]
        for i in range(self.num_layers):
            x = x @ self.weights[i] + self.biases[i]
            x = self.activation_function.forward(x)
            self.features.append(x)
        logits = self.features[-1]
        softmax_x = torch.exp(logits) / torch.sum(torch.exp(logits), dim=1, keepdim=True)
        return softmax_x

    def backward(self, delta: torch.tensor) -> None:
      for i in reversed(range(self.num_layers)):
          # Gradient of weights and biases
          grad_w = self.features[i].T @ delta
          grad_b = delta.mean(dim=0)

          # Update weights and biases using gradient descent
          self.weights[i] -= self.learning_rate * (grad_w + self.l1_reg * torch.sign(self.weights[i]))
          self.biases[i] -= self.learning_rate * grad_b

          # Compute delta for the previous layer (only if it's not the input layer)
          if i > 0:
              # grad_w = self.weights[i]
              # delta = delta @ grad_w.T
              # delta = self.activation_function.backward(delta, self.features[i - 1])

              delta = self.activation_function.backward( (delta @ self.weights[i].T),(self.features[i]))

              # grad_b = delta.mean(dim=0)
              # delta = (delta @ self.weights[i].T) * (self.features[i] > 0).float()  # Apply ReLU derivative

def TrainMLP(model: MLP, x_train: torch.tensor, y_train: torch.tensor) -> MLP:
    bs = model.batch_size
    N = x_train.shape[0]
    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0  # Total loss

    for i in tqdm.tqdm(range(N // bs)):
        x = x_train[idx[i * bs:(i + 1) * bs], ...]
        y = y_train[idx[i * bs:(i + 1) * bs], ...]

        # Forward pass
        y_hat = model.forward(x)

        # Compute cross-entropy loss (convert one-hot to class indices)
        target = torch.argmax(y, dim=1)
        l = -torch.sum(y * torch.log(y_hat))
        L += l.item()

        # Backward pass
        delta = y_hat - y  # Gradient for softmax + cross-entropy
        model.backward(delta)

    print("Train Loss:", L / ((N // bs) * bs))

def TestMLP(model: MLP, x_test: torch.tensor, y_test: torch.tensor) -> tuple[float, float]:
    bs = model.batch_size
    N = x_test.shape[0]

    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0
    A = 0

    for i in tqdm.tqdm(range(N // bs)):
        x = x_test[idx[i * bs:(i + 1) * bs], ...]
        y = y_test[idx[i * bs:(i + 1) * bs], ...]

        y_hat = model.forward(x)
        target = torch.argmax(y, dim=1)

        # Cross-entropy loss
        l = torch.nn.functional.cross_entropy(y_hat, target)
        L += l.item()

        # Accuracy calculation
        predictions = torch.argmax(y_hat, dim=1)
        A += torch.sum(predictions == target).item()

    print("Test Loss:", L / ((N // bs) * bs), "Test Accuracy: {:.2f}%".format(100 * A / N))

def normalize_mnist() -> tuple[torch.tensor, torch.tensor, torch.tensor, torch.tensor]:
    base_path = "/content/drive/MyDrive/Neural_Networks/Data/"

    mnist = MnistDataloader(base_path + "train-images.idx3-ubyte", base_path + "train-labels.idx1-ubyte",
                            base_path + "t10k-images.idx3-ubyte", base_path + "t10k-labels.idx1-ubyte")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_mean = torch.mean(x_train, dim=0, keepdim=True)
    x_std = torch.std(x_train, dim=0, keepdim=True)

    x_train -= x_mean
    x_train /= x_std
    x_train[x_train != x_train] = 0

    x_test -= x_mean
    x_test /= x_std
    x_test[x_test != x_test] = 0

    return x_train, y_train, x_test, y_test

def main():
    x_train, y_train, x_test, y_test = normalize_mnist()

    model = MLP([784, 256, 10])
    model.initialize()
    model.set_hp(lr=1e-6, bs=512, activation=ReLU(), l1_reg=0.01) # Pass l1_reg as an argument

    E = 25
    for _ in range(E):
        TrainMLP(model, x_train, y_train)
        TestMLP(model, x_test, y_test)

if __name__ == "__main__":
    main()



100%|██████████| 117/117 [00:01<00:00, 91.75it/s]


Train Loss: 2.4150270885891385


100%|██████████| 19/19 [00:00<00:00, 165.14it/s]


Test Loss: nan Test Accuracy: 17.93%


100%|██████████| 117/117 [00:01<00:00, 90.86it/s]


...

Test Loss: nan Test Accuracy: 81.89%


100%|██████████| 117/117 [00:01<00:00, 90.43it/s]


Train Loss: 0.671353425225641


100%|██████████| 19/19 [00:00<00:00, 180.32it/s]

Test Loss: nan Test Accuracy: 82.08%


# L2 Regulization

In [ ]:
import torch
import numpy as np
import tqdm

class MLP:
    '''
    This class implements a generic MLP learning framework. The core structure is provided,
    but key functions such as initialize(), forward(), backward(), and TrainMLP() have been completed.
    '''
    def __init__(self, layer_sizes: list[int]):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        self.weights = []
        self.biases = []
        self.features = []

        # Hyperparameters with default values
        self.learning_rate = 1e-6
        self.batch_size = 512
        self.epoch = 25
        self.activation_function = ReLU()

    def set_hp(self, lr: float, bs: int, activation: object, l2_reg: float = 0.0) -> None: # Added l2_reg with a default value
        self.learning_rate = lr
        self.batch_size = bs
        self.activation_function = activation
        self.l2_reg = l2_reg # Assign l2_reg to the model

    def initialize(self) -> None:
        # Initialize weights with uniform distribution +/- sqrt(6 / (d_in + d_out))
        for i in range(self.num_layers):
            d_in, d_out = self.layer_sizes[i], self.layer_sizes[i + 1]
            bound = np.sqrt(6 / (d_in + d_out))
            self.weights.append(torch.empty((d_in, d_out)).uniform_(-bound, bound))
            #self.weights.append(torch.rand((d_in, d_out)) * 2 * bound - bound)
            self.biases.append(torch.zeros(1, d_out))

    def forward(self, x: torch.tensor) -> torch.tensor:
        # Forward propagation
        self.features = [x]
        for i in range(self.num_layers):
            x = x @ self.weights[i] + self.biases[i]
            x = self.activation_function.forward(x)
            self.features.append(x)
        logits = self.features[-1]
        softmax_x = torch.exp(logits) / torch.sum(torch.exp(logits), dim=1, keepdim=True)
        return softmax_x

    def backward(self, delta: torch.tensor) -> None:
      for i in reversed(range(self.num_layers)):
          # Gradient of weights and biases
          grad_w = self.features[i].T @ delta
          grad_b = delta.mean(dim=0)

          # Update weights and biases using gradient descent
          self.weights[i] -= self.learning_rate * (grad_w + self.l2_reg * torch.sign(self.weights[i]))
          self.biases[i] -= self.learning_rate * grad_b

          # Compute delta for the previous layer (only if it's not the input layer)
          if i > 0:
              # grad_w = self.weights[i]
              # delta = delta @ grad_w.T
              # delta = self.activation_function.backward(delta, self.features[i - 1])

              delta = self.activation_function.backward( (delta @ self.weights[i].T),(self.features[i]))

              # grad_b = delta.mean(dim=0)
              # delta = (delta @ self.weights[i].T) * (self.features[i] > 0).float()  # Apply ReLU derivative

def TrainMLP(model: MLP, x_train: torch.tensor, y_train: torch.tensor) -> MLP:
    bs = model.batch_size
    N = x_train.shape[0]
    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0  # Total loss

    for i in tqdm.tqdm(range(N // bs)):
        x = x_train[idx[i * bs:(i + 1) * bs], ...]
        y = y_train[idx[i * bs:(i + 1) * bs], ...]

        # Forward pass
        y_hat = model.forward(x)

        # Compute cross-entropy loss (convert one-hot to class indices)
        target = torch.argmax(y, dim=1)
        l = -torch.sum(y * torch.log(y_hat))
        L += l.item()

        # Backward pass
        delta = y_hat - y  # Gradient for softmax + cross-entropy
        model.backward(delta)

    print("Train Loss:", L / ((N // bs) * bs))

def TestMLP(model: MLP, x_test: torch.tensor, y_test: torch.tensor) -> tuple[float, float]:
    bs = model.batch_size
    N = x_test.shape[0]

    rng = np.random.default_rng()
    idx = rng.permutation(N)

    L = 0
    A = 0

    for i in tqdm.tqdm(range(N // bs)):
        x = x_test[idx[i * bs:(i + 1) * bs], ...]
        y = y_test[idx[i * bs:(i + 1) * bs], ...]

        y_hat = model.forward(x)
        target = torch.argmax(y, dim=1)

        # Cross-entropy loss
        l = torch.nn.functional.cross_entropy(y_hat, target)
        L += l.item()

        # Accuracy calculation
        predictions = torch.argmax(y_hat, dim=1)
        A += torch.sum(predictions == target).item()

    print("Test Loss:", L / ((N // bs) * bs), "Test Accuracy: {:.2f}%".format(100 * A / N))

def normalize_mnist() -> tuple[torch.tensor, torch.tensor, torch.tensor, torch.tensor]:
    base_path = "/content/drive/MyDrive/Neural_Networks/Data/"

    mnist = MnistDataloader(base_path + "train-images.idx3-ubyte", base_path + "train-labels.idx1-ubyte",
                            base_path + "t10k-images.idx3-ubyte", base_path + "t10k-labels.idx1-ubyte")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_mean = torch.mean(x_train, dim=0, keepdim=True)
    x_std = torch.std(x_train, dim=0, keepdim=True)

    x_train -= x_mean
    x_train /= x_std
    x_train[x_train != x_train] = 0

    x_test -= x_mean
    x_test /= x_std
    x_test[x_test != x_test] = 0

    return x_train, y_train, x_test, y_test

def main():
    x_train, y_train, x_test, y_test = normalize_mnist()

    model = MLP([784, 256, 10])
    model.initialize()
    model.set_hp(lr=1e-6, bs=512, activation=ReLU(), l2_reg=0.01) # Pass l2_reg as an argument

    E = 25
    for _ in range(E):
        TrainMLP(model, x_train, y_train)
        TestMLP(model, x_test, y_test)

if __name__ == "__main__":
    main()



100%|██████████| 117/117 [00:01<00:00, 93.10it/s]


Train Loss: 2.3131020904606223


100%|██████████| 19/19 [00:00<00:00, 181.96it/s]


Test Loss: nan Test Accuracy: 23.64%


100%|██████████| 117/117 [00:01<00:00, 88.56it/s]


...

Test Loss: nan Test Accuracy: 82.24%


100%|██████████| 117/117 [00:01<00:00, 90.75it/s]


Train Loss: 0.6516790858700744


100%|██████████| 19/19 [00:00<00:00, 180.00it/s]

Test Loss: nan Test Accuracy: 82.35%
